## Importing the Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/customer_data.csv')

### Checking for null values

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   1125 non-null   int64  
 1   id      1125 non-null   int64  
 2   fea_1   1125 non-null   int64  
 3   fea_2   976 non-null    float64
 4   fea_3   1125 non-null   int64  
 5   fea_4   1125 non-null   float64
 6   fea_5   1125 non-null   int64  
 7   fea_6   1125 non-null   int64  
 8   fea_7   1125 non-null   int64  
 9   fea_8   1125 non-null   int64  
 10  fea_9   1125 non-null   int64  
 11  fea_10  1125 non-null   int64  
 12  fea_11  1125 non-null   float64
dtypes: float64(3), int64(10)
memory usage: 114.4 KB


In [ ]:
dataset.head()

,label,id,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,fea_9,fea_10,fea_11
0,1,54982665,5,1245.5,3,77000.0,2,15,5,109,5,151300,244.948974
1,0,59004779,4,1277.0,1,113000.0,2,8,-1,100,3,341759,207.173840
2,0,58990862,7,1298.0,1,110000.0,2,11,-1,101,5,72001,1.000000
3,1,58995168,7,1335.5,1,151000.0,2,11,5,110,3,60084,1.000000
4,0,54987320,7,NaN,2,59000.0,2,11,5,108,4,450081,197.403141


### We have 1 column that contains null entries, so we will be replacing the null entries with mean

### We will be using skLearn SimpleImputer

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'mean')

In [ ]:
dataset['fea_2'] = imputer.fit_transform(dataset[['fea_2']])


## Stratified K-fold Cross Validation with SVC classifier

In [ ]:
from sklearn.svm import SVC
SVC_model=SVC()

In [ ]:
x=dataset.drop('label', axis=1)
y=dataset['label']

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
SVC_model=SVC(kernel='linear', max_iter=100, random_state=42)
KFoldVal=KFold(10)

In [ ]:
cross_val_score,
result_KFold=cross_val_score(SVC_model,x,y,cv=KFoldVal)
print(result_KFold)

[0.32743363 0.69911504 0.63716814 0.61946903 0.28318584 0.40178571
 0.60714286 0.66071429 0.38392857 0.61607143]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).

In [ ]:
KFold_Score=np.mean(result_KFold)
print(KFold_Score)

0.5236014538558786


## Repeated Random SubSampling with SVC classifier

In [ ]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
SVC_model=SVC(kernel='linear', max_iter=100, random_state=42)
ssplit=ShuffleSplit(n_splits=10, test_size=.15)
result_ssplit=cross_val_score(SVC_model,x,y,cv=ssplit)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).

In [ ]:
result_ssplit

array([0.64497041, 0.66863905, 0.61538462, 0.56213018, 0.62721893,
       0.36686391, 0.21893491, 0.65088757, 0.60946746, 0.62130178])

In [ ]:
ssplit_score=np.mean(result_ssplit)
print(ssplit_score)

0.5585798816568047


## Randomized Search with Logistic Regressor

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.15, random_state=42)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
param={'C':[0,1,1,10,100], 'solver':['liblinear', 'saga', 'lbfgs'], 'max_iter':[100,1000,10000], 'random_state':[10, 50, 100]}

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
random_model=RandomizedSearchCV(LogisticRegression(), param, verbose=3, n_iter=3)
random_model.fit(x_train, y_train)
random_model.best_params_
random_model.best_score_

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END C=0, max_iter=1000, random_state=100, solver=saga;, score=nan total time=   0.0s
[CV 2/5] END C=0, max_iter=1000, random_state=100, solver=saga;, score=nan total time=   0.0s
[CV 3/5] END C=0, max_iter=1000, random_state=100, solver=saga;, score=nan total time=   0.0s
[CV 4/5] END C=0, max_iter=1000, random_state=100, solver=saga;, score=nan total time=   0.0s
[CV 5/5] END C=0, max_iter=1000, random_state=100, solver=saga;, score=nan total time=   0.0s
[CV 1/5] END C=100, max_iter=100, random_state=10, solver=liblinear;, score=0.807 total time=   0.0s
[CV 2/5] END C=100, max_iter=100, random_state=10, solver=liblinear;, score=0.806 total time=   0.0s
[CV 3/5] END C=100, max_iter=100, random_state=10, solver=liblinear;, score=0.806 total time=   0.0s
[CV 4/5] END C=100, max_iter=100, random_state=10, solver=liblinear;, score=0.806 total time=   0.0s
[CV 5/5] END C=100, max_iter=100, random_state=10, solver=liblinea

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1614, in fit
    for class_, warm_start_coef_ in zip(classes_, warm_start_coef)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 1085, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/

0.8064845113438045

In [ ]:
random_model.best_params_
random_model.best_score_

0.8064845113438045

## Grid Search with SVC classifier

In [ ]:
from sklearn.model_selection import GridSearchCV
param={'C':[0,10,100], 'kernel':['rbf', 'linear', 'lbfgs'], 'gamma':[0.1,0.01, 0.001]}

In [ ]:
grid_model=GridSearchCV(SVC(), param, verbose=2, cv=2)
grid_model.fit(x_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time=   0.1s
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time=   0.1s


In [ ]:
grid_model.best_params_
grid_model.best_score_

## Visualizing the results

In [ ]:
from prettytable import PrettyTable
myTable=PrettyTable(['Technique used', 'Score'])
myTable.add_row(['Stratified K-fold Cross Validation with SVC classifier', result_KFold])
myTable.add_row(['Repeated Random SubSampling with SVC classifier', ssplit_score])
myTable.add_row(['Randomized Search with Logistic Regressor', random_model.best_score_])
myTable.add_row(['Grid Search with SVC classifier', grid_model.best_score_])